# GPT4All

[GitHub:nomic-ai/gpt4all](https://github.com/nomic-ai/gpt4all) 是一个开源聊天机器人生态系统，它在一个包含代码、故事和对话的海量干净的助手数据集上进行了训练。

本示例将介绍如何使用 LangChain 与 `GPT4All` 模型进行交互。

In [ ]:
%pip install --upgrade --quiet langchain-community gpt4all

### 导入 GPT4All

In [2]:
from langchain_community.llms import GPT4All
from langchain_core.prompts import PromptTemplate

### 设置要传递给 LLM 的问题

In [3]:
template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate.from_template(template)

### 指定模型

要在本地运行，请下载兼容的 ggml 格式模型。

[gpt4all 页面](https://gpt4all.io/index.html)有一个有用的 `模型浏览器` 部分：

* 选择感兴趣的模型
* 使用 UI 下载并将 `.bin` 文件移动到 `local_path`（如下文所述）

有关更多信息，请访问 https://github.com/nomic-ai/gpt4all。

---

此集成尚不支持通过 `.stream()` 方法以块的形式进行流式传输。下面的示例使用带有 `streaming=True` 的回调处理程序：

In [4]:
local_path = (
    "./models/Meta-Llama-3-8B-Instruct.Q4_0.gguf"  # replace with your local file path
)

In [10]:
from langchain_core.callbacks import BaseCallbackHandler

count = 0


class MyCustomHandler(BaseCallbackHandler):
    def on_llm_new_token(self, token: str, **kwargs) -> None:
        global count
        if count < 10:
            print(f"Token: {token}")
            count += 1


# Verbose is required to pass to the callback manager
llm = GPT4All(model=local_path, callbacks=[MyCustomHandler()], streaming=True)

# If you want to use a custom model add the backend parameter
# Check https://docs.gpt4all.io/gpt4all_python.html for supported backends
# llm = GPT4All(model=local_path, backend="gptj", callbacks=callbacks, streaming=True)

chain = prompt | llm

question = "What NFL team won the Super Bowl in the year Justin Bieber was born?"

# Streamed tokens will be logged/aggregated via the passed callback
res = chain.invoke({"question": question})

Token:  Justin
Token:  Bieber
Token:  was
Token:  born
Token:  on
Token:  March
Token:  
Token: 1
Token: ,
Token:  
